In [1]:
from ultralytics import YOLO
import os

# Get the absolute path to your dataset YAML file
DATASET_YAML = "D:\Github\MachineLearningFinalProject\dataset.yaml"

# Initialize the model
model = YOLO("yolov5nu.pt")

# Train the model using your custom dataset
results = model.train(
    data=DATASET_YAML,  # Path to your dataset YAML file
    epochs=100,         # Number of training epochs
    imgsz=640,         # Image size
    batch=64,          # Batch size
    name='yolov5_custom'  # Name for the experiment
)

Ultralytics 8.3.40  Python-3.11.9 torch-2.5.1+cpu CPU (AMD Ryzen 9 9950X 16-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov5nu.pt, data=D:\Github\MachineLearningFinalProject\dataset.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov5_custom8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

train: Scanning D:\Github\MachineLearningFinalProject\dataset\train... 11247 images, 12 backgrounds, 0 corrupt: 100%|██████████| 11259/11259 [00:05<00:00, 1912.20it/s]


train: New cache created: D:\Github\MachineLearningFinalProject\dataset\train.cache


val: Scanning D:\Github\MachineLearningFinalProject\dataset\valid... 1877 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1879/1879 [00:01<00:00, 1424.81it/s]


val: New cache created: D:\Github\MachineLearningFinalProject\dataset\valid.cache
Plotting labels to d:\Github\MachineLearningFinalProject\runs\detect\yolov5_custom8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to d:\Github\MachineLearningFinalProject\runs\detect\yolov5_custom8
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      1.351      4.338       1.57        249        640:   7%|▋         | 12/176 [01:33<21:23,  7.83s/it]


KeyboardInterrupt: 

In [ ]:
import os
import random
from pathlib import Path

# Create output directory if it doesn't exist
output_dir = "inference_output"
os.makedirs(output_dir, exist_ok=True)

# Get a random image from your test dataset
test_dir = os.path.join(DATASET_PATH, "test")
test_images = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
random_image = os.path.join(test_dir, random.choice(test_images))

# Run inference
results = model.predict(
    source=random_image,
    save=True,              # Save results
    project=output_dir,     # Output directory
    name='random_test',     # Name of the run
    conf=0.25,             # Confidence threshold
    save_txt=True          # Save labels
)

print(f"Processed image: {random_image}")
print(f"Results saved to: {output_dir}/random_test")